<a href="https://colab.research.google.com/github/solegas/GAN/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
import os
import numpy as np
import torch
import torchvision
import torch.optim as opt 
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import save_image
import matplotlib.pyplot as plt 
%matplotlib inline


In [0]:
#parameters 
#number of epochs
n_epochs = 100
#batch size
batch_size = 2
#learning rate
lr = 0.0002

#create a path for the folder with the saved output
folder = "My\ Drive/Thesis/imagesGAN"
if not os.path.exists(folder):
    os.makedirs(folder)

#transform a numpy array to pytorch tensor
transform=transforms.ToTensor()

#download data for training and transform in tensor form
#trainData = torchvision.datasets.MNIST('./data/', download = True, transform = transform, train= True)
trainData = np.append([[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0]],
                             [[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                              [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0]],axis=0)

trainData = torch.from_numpy(trainData).float()
#initialize dataloader in order to train on a random batch size from the whole dataset at a time to the neural network
trainLoader = torch.utils.data.DataLoader(trainData, shuffle = True, batch_size = batch_size)

#function to iterate over data feeded by trainLoader and get the actual "image" and "label" from it
dataIter=iter(trainLoader)
imgs = dataIter.next()

In [0]:
#verify the shape of imgs
imgs.shape

torch.Size([2, 24])

In [0]:
#visualize imgs 
def imshow(imgs): 
  print(imgs)

In [0]:
imshow(imgs)


tensor([[0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 0., 0., 0., 0.]])


In [0]:
#noise dimension (arbitrary)
Z_dim = 10

#hidden dimension nodes
H_dim = 100 

#input dimension (total number of pixels of an image 28*28=784)
X_dim = 24

In [0]:
#define generator
class Gen(nn.Module): #inherit functions from pytorch module library
    def __init__(self): #initialize the instances of the class in order to access its attributes and methods
        super().__init__() #super() to make other classes inherit the generators attributes
        self.model = nn.Sequential(     #sequential container to define layers, here we defined a minimum number of layers
            nn.Linear(Z_dim, H_dim),    #linear layer with input size of the noise and output size of the hidden layer
            nn.ReLU(),                  #activation function
            nn.Linear(H_dim,X_dim),     #second linear layer that goes from hidden layer to output layer X
            nn.Sigmoid()                #sigmoid in order to have a probability 
        )

    def forward(self, input):         #takes the input and makes it run through the self.model defined above
        return self.model(input)


In [0]:

G = Gen() #initialize the generator

In [0]:
#define discriminator
class Dis(nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.model = nn.Sequential(     
            nn.Linear(X_dim, H_dim),    #linear layer with input size of X and output size of the hidden layer
            nn.ReLU(),                  
            nn.Linear(H_dim,1),     #second linear layer that goes from hidden layer to 1(true or false)
            nn.Sigmoid()                
        )

    def forward(self, input):         
        return self.model(input)

In [0]:
D = Dis()  #initialize the discriminator

In [0]:
print(G)
print(D)

Gen(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=24, bias=True)
    (3): Sigmoid()
  )
)
Dis(
  (model): Sequential(
    (0): Linear(in_features=24, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


In [0]:
#Define optimizers for each network (Adam)
g_opt = opt.Adam(G.parameters(), lr = lr)
d_opt = opt.Adam(D.parameters(), lr = lr)

In [0]:
#Define training loop
for epoch in range(n_epochs):
  G_loss_run = 0.0
  D_loss_run = 0.0
  for i, data in enumerate(trainLoader):           #take every batch one by one
    X = data                                    #store data in X
    #X = X.view(X.size(0), -1)                      #convert data 
    batch_size = X.size(0)                         #if last batch is smaller than batch_size

    one_labels = torch.ones(batch_size, 1)         #define array of ones
    zero_labels = torch.zeros(batch_size, 1)       #define array of zeros

    z = torch.randn(batch_size, Z_dim)             #create random noise from normal distribution

    D_real = D(X)                                  #pass real data through discriminator
    D_fake = D(G(z))                               #pass generated data by the generator through discriminator

    D_real_loss = F.binary_cross_entropy(D_real,one_labels)   #loss from real data that should be ones after the discriminator
    D_fake_loss = F.binary_cross_entropy(D_fake,zero_labels) #loss from fake data that should be zeros after the discriminator

    D_loss = D_real_loss + D_fake_loss            #total loss

    d_opt.zero_grad()                             #since pytorch accumulates the gradients we need to reset it to 0 every loop
    D_loss.backward()                             #compute gradients 
    d_opt.step()                                  #update weights and biases for discriminator with the set learning rate

    z = torch.randn(batch_size, Z_dim)             #recreate random noise
    D_fake = D(G(z))                               #pass generated data by the generator through discriminator
    G_loss = F.binary_cross_entropy(D_fake, one_labels)      #loss from fake data that should fool the discriminator and be ones

    g_opt.zero_grad()
    G_loss.backward()
    g_opt.step()                                  #update weights and biases for generator with the set learning rate

    G_loss_run += G_loss.item() 
    D_loss_run += D_loss.item()      

  print('Epoch:{},   G_loss:{},   D_loss:{}'.format(epoch, G_loss_run/(i+1), D_loss_run/(i+1)))
  samples = G(z).detach()    #detach to stop from automatic backpropagation
  #samples = samples.view(samples.size(0),1, 28, 28) #reshape from output of G to image format
  imshow(samples)




Epoch:0,   G_loss:0.7896425366401673,   D_loss:1.4031760215759277
tensor([[0.4563, 0.5435, 0.3789, 0.3967, 0.5653, 0.5061, 0.4089, 0.5473, 0.5737,
         0.4490, 0.4018, 0.4334, 0.6041, 0.5120, 0.3745, 0.3802, 0.4030, 0.4307,
         0.4982, 0.6096, 0.6854, 0.4441, 0.5540, 0.5620],
        [0.5012, 0.5413, 0.5017, 0.4299, 0.4747, 0.5791, 0.4408, 0.5599, 0.3689,
         0.4124, 0.5090, 0.5221, 0.4676, 0.6575, 0.5224, 0.5085, 0.4207, 0.4234,
         0.4274, 0.4766, 0.7024, 0.4460, 0.5976, 0.5694]])
Epoch:1,   G_loss:0.791015899181366,   D_loss:1.3770414590835571
tensor([[0.5518, 0.6053, 0.5119, 0.4714, 0.4378, 0.4467, 0.5471, 0.6063, 0.4762,
         0.4428, 0.4954, 0.6374, 0.5493, 0.5937, 0.5115, 0.4295, 0.4358, 0.3985,
         0.4422, 0.5098, 0.6909, 0.5031, 0.4772, 0.5545],
        [0.4693, 0.5247, 0.4992, 0.3776, 0.4937, 0.5026, 0.5394, 0.6294, 0.4053,
         0.5261, 0.3777, 0.4535, 0.4755, 0.5032, 0.4206, 0.4989, 0.5173, 0.5125,
         0.3838, 0.4314, 0.6411, 0.4852, 0.531